In [ ]:
notebook_id = ''
git_repo = ''
branch = ''
target_file = ''
package_management_file = ''
docker_registry = ''
docker_repo = f'{docker_registry}/training'
image_version = ''
image_tag = f'{notebook_id}_{image_version}'

In [ ]:
assert notebook_id
assert git_repo
assert branch
assert target_file
assert docker_registry
assert docker_repo
assert image_tag

In [ ]:
install_packages_cmd = ''
if package_management_file:
    install_packages_cmd = f'''
    && cat {package_management_file} \\
    && while read requirement; do conda run -n ${{runtime}} conda install -y ${{requirement}} || conda run -n ${{runtime}} pip install --user ${{requirement}}; done < {package_management_file}
'''

dockerfile = f'''
FROM gn00023040/notebook-runtime:1.0

RUN set -xe \\
    && git clone --single-branch -b master {git_repo} repo \\
    && cd repo \\
    && lang=$(cat {target_file} | jq -r '.metadata.language_info.name') \\
    && lang_ver=$(cat {target_file} | jq -r '.metadata.language_info.version') \\
    && runtime=${{lang}}_${{lang_ver}} \\
    && kernel=$(cat {target_file} | jq -r '.metadata.kernelspec.name') \\
    && conda create -y -n ${{runtime}} ${{lang}}=${{lang_ver}} \\
    && conda run -n ${{runtime}} python --version \\
    && conda run -n ${{runtime}} conda install -y ipykernel \\
    && conda run -n ${{runtime}} python -m ipykernel install --user --name=${{kernel}} \\
    {install_packages_cmd}
'''

with open('Dockerfile', 'w') as file:
    file.write(dockerfile)

In [ ]:
!set -xe \
    && cat Dockerfile \
    && sudo docker build -t {docker_repo}:{image_tag} . \
    && aws ecr get-login-password --region us-east-2 | sudo docker login --username AWS --password-stdin {docker_registry} \
    && sudo docker push {docker_repo}:{image_tag}